In [1]:
pip install opencv-python numpy streamlit pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 656.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00


In [2]:
pip install gradio tensorflow matplotlib SimpleITK


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00


In [7]:
import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

# Preprocessing function (grayscale, noise removal)
def preprocess_image(image, filter_size):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(gray_image, (filter_size, filter_size), 0)

    return blurred_image

# Thresholding using Otsu's method
def otsu_thresholding(image):
    _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded_image

# Manual thresholding function
def manual_thresholding(image, threshold_value):
    _, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded_image

# Morphological operations to clean the segmentation
def morphological_operations(image):
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)  # Closing: Fill small holes
    opened_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)   # Opening: Remove noise
    return opened_image

# Feature extraction (Find contours)
def extract_contours(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# Function to visualize the segmentation process
def segment_tumor(image, filter_size, threshold_value):
    # Preprocess the image
    blurred_image = preprocess_image(image, filter_size)

    # Apply thresholding (Otsu or manual)
    thresholded_image = otsu_thresholding(blurred_image)  # You can replace this with manual_thresholding

    # Perform morphological operations to clean the image
    cleaned_image = morphological_operations(thresholded_image)

    # Find contours
    contours = extract_contours(cleaned_image)

    # Create a blank image to display the segmented tumor
    segmented_image = np.zeros_like(image)
    cv2.drawContours(segmented_image, contours, -1, (0, 255, 0), 2)  # Draw green contours

    # Display original and segmented image side by side
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(image)
    ax[0].set_title("Original MRI")
    ax[1].imshow(segmented_image)
    ax[1].set_title("Tumor Segmentation")
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    return fig

# Gradio interface
interface = gr.Interface(fn=segment_tumor,
                         inputs=[gr.Image(type="numpy", image_mode="RGB"),
                                 gr.Slider(3, 15, step=2, label="Filter Size", value=5), # Changed here
                                 gr.Slider(0, 255, step=1, label="Threshold Value", value=128)], # Changed here
                         outputs=gr.Plot(),
                         live=True)


# Launch the Gradio interface
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://00d325e338cfe6294b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
